# Test basic classification models for topic segmentation

In [160]:
import model.load_data as ld
import model.scoring_metrics as sm
import model_trainer_and_tester as mtt
import importlib
importlib.reload(sm)
importlib.reload(ld)
importlib.reload(sm)
importlib.reload(mtt)

import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
from itertools import combinations

from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

import sys
np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_columns', None)


## Splits

In [154]:
datasets = """Bed002 Bed003 Bed004 Bed005 Bed006 Bed008 Bed009 Bed010 Bed011 Bed012 Bed013 Bed014 Bed015 Bed016 Bed017 Bmr001 Bmr002 Bmr005 Bmr007 Bmr009 Bmr010 Bmr011 Bmr012 Bmr013 Bmr014 Bmr018 Bmr019 Bmr021 Bmr022 Bmr024 Bmr025 Bmr026 Bmr027 Bmr029 Bns001 Bns002 Bns003 Bro003 Bro004 Bro005 Bro007 Bro008 Bro010 Bro011 Bro012 Bro013 Bro014 Bro015 Bro016 Bro017 Bro018 Bro019 Bro021 Bro022 Bro023 Bro024 Bro025 Bro026 Bro027 Bro028 Bsr001 Btr001 Btr002""".split(" ")
results_merged_path = "../results_merged_f0_stds_fixed/"

all_features = ['pause', 'speakerChange', 'similarity', 'f0_diff', 'f0_baseline_diff']
features_selected = ['pause', 'speakerChange', 'similarity', 'f0_diff', 'f0_baseline_diff']
split=0.3

# Represents the context that is being used for training and for evaluation
shifts = [-3,-2,-1, 1,2,3]

subtopic_lvl = 1

### Basic random split

In [66]:
X_train, y_train, X_test, y_test = ld.train_test_split(datasets,results_merged_path,split)

### Basic random split + context

In [161]:
X_train, y_train, X_test, y_test = mtt.read_in_dataset_all_together(features_selected,shifts,split,subtopic_lvl)

### Constant split + context (to compare with biLSTM)

In [58]:
X_train_const, y_train_const = mtt.read_in_dataset(features_selected, shifts, to_read='train')

clf = DecisionTreeClassifier()
clf.fit(X_train_const, y_train_const)

results_const = mtt.test_set_evaluate_multiple(clf, features_selected, shifts)

In [59]:
results_const.mean()

Pk         0.572100
K-k       -0.066113
Windiff    0.572100
dtype: float64

## Test on commonly used models for topic segmentation

In [167]:
def print_eval(y_pred,y_true):
    k = int(max(1,np.floor((len(y_true)+1)/(2*(sum(y_true)+1)))))
    print('k =',k)

    print('- windiff:',sm.get_windiff(np.array(y_true),np.array(y_pred),k))
    print('- pk:',sm.get_pk(np.array(y_true),np.array(y_pred),k))
    print('- kkappa:',sm.get_k_kappa(np.array(y_true),np.array(y_pred),k))

### Decision Tree classifier with/without hyperparameter tuning

In [166]:
def DecTree(X_train, X_test, y_train,tuning=False, best_criterion=None,best_max_depth=None,best_min_sample_leaf=None):
    if tuning:
        clf = DecisionTreeClassifier(criterion=best_criterion,max_depth=best_max_depth,min_samples_leaf=best_min_sample_leaf)
    else:
        clf = DecisionTreeClassifier()
    clf.fit(X_train, y_train)
    y_predicted = clf.predict(X_test)

    return y_predicted

In [62]:
def DecTree_hyperparam(X,y):
    
    clf = DecisionTreeClassifier()
    std_slc = StandardScaler()
    pipe = Pipeline(steps=[('std_slc', std_slc),('dec_tree', clf)])

    n_components = list(range(1,X.shape[1]+1,1))

    criterion = ['gini', 'entropy','log_loss']
    max_depth = [2,5,10,15,20,30,50,100]
    min_samples_leaf=[5,10,20,50,100]

    parameters = dict(dec_tree__criterion=criterion,
                      dec_tree__max_depth=max_depth,
                      dec_tree__min_samples_leaf=min_samples_leaf)

    clf_GS = GridSearchCV(pipe, parameters)
    clf_GS.fit(X, y)

    best_criterion = clf_GS.best_estimator_.get_params()['dec_tree__criterion']
    best_max_depth = clf_GS.best_estimator_.get_params()['dec_tree__max_depth']
    best_min_samples_leaf = clf_GS.best_estimator_.get_params()['dec_tree__min_samples_leaf']

    print('Best criterion:', best_criterion)
    print('Best max_depth:', best_max_depth)
    print('Best min_sample_leaf:', best_min_samples_leaf)

    return best_criterion,best_max_depth,best_min_samples_leaf

In [60]:
best_criterion,best_max_depth,best_min_sample_leaf = DecTree_hyperparam(X_train,y_train)

print("-------------Tuned DST")
DT_y_predicted = DecTree(X_train,X_test,y_train,True,best_criterion,best_max_depth,best_min_sample_leaf)
print(sum(DT_y_predicted))
print_eval(DT_y_predicted,y_test)

NameError: name 'DecTree_hyperparam' is not defined

In [168]:
print("-------------Normal DST")
DT_y_predicted = DecTree(X_train,X_test,y_train,False)
print(sum(DT_y_predicted),sum(y_test))
print_eval(DT_y_predicted,y_test)
indexes = [index for index in range(len(DT_y_predicted)) if (DT_y_predicted[index]==1)and(y_test[index]==1)]

print(len(indexes))


-------------Normal DST
359 299
k = 42
- windiff: 0.5457196909979505
- pk: 0.47773135740186035
- kkappa: 0.008510417968356592
13


### SVM with/without hyperparameter tuning

In [94]:
def SVM(X_train,X_test,y_train,tuning=False,best_gamma=None,best_C=None):
    if tuning:
        clf = SVC(gamma=best_gamma,C=best_C) 
    else:
        clf = SVC() 
    clf.fit(X_train, y_train) 
    y_predicted = clf.predict(X_test)
    
    return y_predicted

In [ ]:
def SVM_hyperparam(X,y):
    param_grid = {
        "gamma": [0.1, 1.0, 10],
        "C": [0.1, 1.0, 10]
    }

    grid_search = GridSearchCV(SVC(), param_grid, cv=2, verbose=1, n_jobs=-1)
    grid_search.fit(X, y)

    best_gamma = grid_search.best_estimator_.get_params()['gamma']
    best_C = grid_search.best_estimator_.get_params()['C']
    print('Best gamma:', best_gamma)
    print('Best C:', best_C)

    return best_gamma,best_C

In [ ]:
best_gamma,best_C = SVM_hyperparam(X_train,y_train)

print("-------------Tuned SVM")
SVM_y_predicted = SVM(X_train,X_test,y_train,True,best_gamma,best_C)
print(sum(SVM_y_predicted))
print_eval(SVM_y_predicted,y_test)

C:\Users\marti\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 2 folds for each of 9 candidates, totalling 18 fits
Best gamma: 0.1
Best C: 0.1


In [68]:
print("-------------Normal SVM")
SVM_y_predicted = SVM(X_train,X_test,y_train,False)
print(sum(SVM_y_predicted),sum(y_test))
print_eval(SVM_y_predicted,y_test)

-------------Normal SVM
0.0
k = 67
- windiff: 0.3934795902767161
- pk: 0.3934795902767161
- kkappa: 0.0


### Naive Bayes for binary class with/without hyperparameter tuning

In [95]:
def naiveBayes(X_train,X_test,y_train,tuning=False,best_var_smoothing=None):

    if tuning:
        clf = GaussianNB(var_smoothing=best_var_smoothing) 
    else:
        clf = GaussianNB() 

    clf.fit(X_train, np.ravel(y_train)) 
    y_predicted = clf.predict(X_test)
    
    return y_predicted

In [ ]:
def naiveBayes_hyperparam(X,y):
        param_grid_nb = {'var_smoothing': np.logspace(0,-9, num=100)}
        nbModel_grid = GridSearchCV(estimator=GaussianNB(), param_grid=param_grid_nb, verbose=1, cv=10, n_jobs=-1)
        nbModel_grid.fit(X, y)

        best_var_smoothing = nbModel_grid.best_estimator_.get_params()['var_smoothing']
        print('Best var_smoothing:', best_var_smoothing)

        return best_var_smoothing

In [70]:
best_var_smoothing = naiveBayes_hyperparam(X_train,y_train)

print("-------------Tuned NB")
NB_y_predicted = naiveBayes(X_train,X_test,y_train,True,best_var_smoothing)
print(sum(NB_y_predicted))
print_eval(NB_y_predicted,y_test)

NameError: name 'naiveBayes_hyperparam' is not defined

In [96]:
print("-------------Normal NB")
NB_y_predicted = naiveBayes(X_train,X_test,y_train,False)
print(sum(NB_y_predicted),sum(y_test))
print_eval(NB_y_predicted,y_test)

-------------Normal NB
480.0 241.0
k = 53
- windiff: 0.5302481399244283
- pk: 0.40302286626933115
- kkappa: 0.14924650743217874


## Multiple runs of each algo with different random splits

In [9]:
def get_eval(y_pred,y_true):
    k = int(max(1,np.floor((len(y_true)+1)/(2*(y_true['boundary'].sum()+1)))))

    windiff = sm.get_windiff(np.array(y_true),np.array(y_pred),k)
    pk = sm.get_pk(np.array(y_true),np.array(y_pred),k)
    kappa = sm.get_k_kappa(np.array(y_true),np.array(y_pred),k)

    return windiff,pk,kappa

def get_avg_eval(eval):
    windiffs = [row[0] for row in eval]
    pks = [row[1] for row in eval]
    kappas = [row[2] for row in eval]

    print('-> windiff:',windiffs)
    print('-> pk:',pks)
    print('-> k-kappa:',kappas)

    # print('-> windiff - mean:',statistics.mean(windiffs),', - var:',statistics.variance(windiffs))
    # print('-> pk - mean:',statistics.mean(pks),', - var:',statistics.variance(pks))
    # print('-> k-kappa - mean:',statistics.mean(kappas),', - var:',statistics.variance(kappas))

### Test different context levels

In [10]:
all_features = ['pause', 'speakerChange', 'similarity', 'f0_diff', 'f0_baseline_diff']
features_selected = ['pause', 'speakerChange', 'similarity', 'f0_diff', 'f0_baseline_diff']

split = 0.3
# Represents the context that is being used for training and for evaluation
shifts = [-10,-9,-8,-7,-6, -5, -4, -3, -2, -1, 1, 2, 3, 4, 5, 6,7,8,9,10]

iterations = 1
print('EXPERIMENTING ON CONTEXT LEVEL WITH SPLIT',split,'(of testing),',iterations,'ITERATIONS PER MODEL TESTING, AND',features_selected,'FEATURES:')
i=0
while i<len(shifts)/2:
    # get current shift
    curr_shifts=[]
    for j in range(i,len(shifts)-i):
        curr_shifts.append(shifts[j])
    
    DT_scores = []
    SVM_scores = []
    NB_scores = []
    for k in range(iterations):
        X_train, y_train, X_test, y_test = mtt.read_in_dataset_all_together(features_selected,curr_shifts,split)
        
        DT_scores.append(get_eval(DecTree(X_train,X_test,y_train,False),y_test))
        # SVM_scores.append(get_eval(SVM(X_train,X_test,y_train,False),y_test))
        NB_scores.append(get_eval(naiveBayes(X_train,X_test,y_train,False),y_test))
    
    print('-> Context level =',len(curr_shifts)/2)
    print('----- DT:')
    get_avg_eval(DT_scores)
    # print('----- SVM:')
    # get_avg_eval(SVM_scores)
    print('----- NB:')
    get_avg_eval(NB_scores)
    print()
        
    i+=1

EXPERIMENTING ON CONTEXT LEVEL WITH SPLIT 0.3 (of testing), 1 ITERATIONS PER MODEL TESTING, AND ['pause', 'speakerChange', 'similarity', 'f0_diff', 'f0_baseline_diff'] FEATURES:
-> Context level = 10.0
----- DT:
-> windiff: [0.38874398442379043]
-> pk: [0.38874398442379043]
-> k-kappa: [0.18201214015265343]
----- NB:
-> windiff: [0.38874398442379043]
-> pk: [0.38874398442379043]
-> k-kappa: [0.18201214015265343]

-> Context level = 9.0
----- DT:
-> windiff: [0.39850910424049857]
-> pk: [0.39850910424049857]
-> k-kappa: [0.16873222267370988]
----- NB:
-> windiff: [0.39850910424049857]
-> pk: [0.39850910424049857]
-> k-kappa: [0.16873222267370988]

-> Context level = 8.0
----- DT:
-> windiff: [0.39033511769450757]
-> pk: [0.39033511769450757]
-> k-kappa: [0.1798773153716623]
----- NB:
-> windiff: [0.39033511769450757]
-> pk: [0.39033511769450757]
-> k-kappa: [0.1798773153716623]

-> Context level = 7.0
----- DT:
-> windiff: [0.38952795118187555]
-> pk: [0.38952795118187555]
-> k-kappa: [

### Test different combinations of features

In [12]:
datasets = """Bed002 Bed003 Bed004 Bed005 Bed006 Bed008 Bed009 Bed010 Bed011 Bed012 Bed013 Bed014 Bed015 Bed016 Bed017 Bmr001 Bmr002 Bmr005 Bmr007 Bmr009 Bmr010 Bmr011 Bmr012 Bmr013 Bmr014 Bmr018 Bmr019 Bmr021 Bmr022 Bmr024 Bmr025 Bmr026 Bmr027 Bmr029 Bns001 Bns002 Bns003 Bro003 Bro004 Bro005 Bro007 Bro008 Bro010 Bro011 Bro012 Bro013 Bro014 Bro015 Bro016 Bro017 Bro018 Bro019 Bro021 Bro022 Bro023 Bro024 Bro025 Bro026 Bro027 Bro028 Bsr001 Btr001 Btr002""".split(" ")
results_merged_path = "../results_merged_f0_stds_fixed/"

all_features = ['pause', 'speakerChange', 'similarity', 'f0_diff', 'f0_baseline_diff']
features_selected = ['pause', 'speakerChange', 'similarity', 'f0_diff', 'f0_baseline_diff']

split = 0.3
iterations = 10

print('EXPERIMENTING ON DTC AND NB WITH SPLIT',split,'(of testing) AND',iterations,'ITERATIONS PER MODEL TESTING')
for i in range(1,len(all_features)+1):
    print('------ALL COMBINATIONS OF LENGTH',i,'------')
    features_combinations = combinations(all_features,i)
    for feature_comb in features_combinations:
        feature_comb = list(feature_comb)
        DT_scores = []
        NB_scores = []
        for j in range(iterations):
            X_train, y_train, X_test, y_test = ld.train_test_split(datasets,results_merged_path,split)

            X_train = X_train[feature_comb]
            X_test = X_test[feature_comb]
            
            DT_scores.append(get_eval(DecTree(X_train,X_test,y_train,False),y_test))
            NB_scores.append(get_eval(naiveBayes(X_train,X_test,y_train,False),y_test))
        
        get_summary(DT_scores,NB_scores,feature_comb)
        print()
    print()



EXPERIMENTING ON DTC AND NB WITH SPLIT 0.3 (of testing) AND 10 ITERATIONS PER MODEL TESTING
------ALL COMBINATIONS OF LENGTH 1 ------
-> Features = ['pause']
----- DT:
-> windiff: [0.3990531459987783, 0.4123570942192596, 0.38942615239887113, 0.41706355723435706, 0.4106026414117125, 0.41493086892169956, 0.4134905900768083, 0.40046127700898276, 0.4257617098681219, 0.4003784903235021]
-> pk: [0.38313225412339647, 0.3988460022219833, 0.3825023518344309, 0.404828379044178, 0.39928230550109134, 0.4107793942456951, 0.40009464526227656, 0.3944727684713118, 0.4164013945732909, 0.3906305791616082]
-> k-kappa: [0.06073296934030339, 0.02866474859545466, 0.024353556506247376, 0.026040824237152815, 0.008755894521889507, 0.001991889565055156, 0.043459358229195444, 0.03931016302271508, 0.013570210378932381, 0.056110065879273086]
----- Naive Bayes:
-> windiff: [0.5290928527794746, 0.48381894419954846, 0.45170272812793977, 0.406429627196584, 0.5246568754393104, 0.5038446265477781, 0.5145062065450839, 0.

#### To read the file of the output of the code 
Please add the file in the project folder

In [8]:
path = (os.path.realpath(os.path.join(os.getcwd(), (f"simple_models.txt"))))
file = open(path, 'r')
lines = file.readlines()

combination=['pause', 'speakerChange', 'similarity', 'f0_diff', 'f0_baseline_diff']

for i in range(len(lines)):
    if str(combination) in lines[i]:
        DT_windiff=lines[i+2][12:-1].strip('][').split(', ')
        DT_pk=lines[i+3][7:-1].strip('][').split(', ')
        DT_kappa=lines[i+4][12:-1].strip('][').split(', ')

        NB_windiff=lines[i+6][12:-1].strip('][').split(', ')
        NB_pk=lines[i+7][7:-1].strip('][').split(', ')
        NB_kappa=lines[i+8][12:-1].strip('][').split(', ')


['0.10658901085020546', '0.10266616909897681', '-0.04659279581705039', '0.029284132290540693', '0.12114758465146477', '0.05468937802760812', '-0.04198029181586795', '0.0025401207878630134', '0.1894085782008286', '0.07655725318534783']
